### Verify the services are running


In [ ]:
#First let us verify that the IRSA backend service is running
!curl -k https://irsa-svc.domino-field/healthz

In [ ]:
!curl http://localhost:6003/healthz

## Add/Update Role Mapping (Only Domino Administrators can make this call)
```
POST https://irsa-svc.domino-field/update_role_mapping
```

This endpoint adds a mapping between a Domino Organization, AWS Role (the user will assume) and the Proxy AWS Role through which the AWS role is assumed.
The payload takes the following form:
```json
{
    "domino_org":"my_test_org",
    "iam_role_arn":"arn:aws:iam::<ASSETS_ACCOUNT_NO>:role/test-role",
    "proxy_iam_role_arn":"arn:aws:iam::<EKS_ACCOUNT_NO>:role/test-role"
}
```
**Note**: The proxy role is required for scaling to a large number of workloads. If the EKS Cluster and aws assets are in the same account, the aws account numbers for the two roles will be the same

**Note**: The endpoint does not check if the organization name is valid. Make sure you add a valid organization (even if after running the endpoint)proxy_iam_role

In [ ]:
#Fetch my mappings (Any user can do this)
import requests
import os
def get_headers():
    access_token_endpoint='http://localhost:8899/access-token'
    token = requests.get(access_token_endpoint).text
    headers = {
             "Content-Type": "application/json",
             "Authorization": "Bearer " + token,
     }
    return headers

In [ ]:
import requests
import os

def add_org_to_role_mapping(eks_account_no,assets_account_no,domino_org_name,asset_role_name,proxy_role_name):
    endpoint='https://irsa-svc.domino-field/update_role_mapping'
    body={
        "domino_org":f"{domino_org_name}",
        "iam_role_arn":f"arn:aws:iam::{assets_account_no}:role/{asset_role_name}",
        "proxy_iam_role_arn":f"arn:aws:iam::{eks_account_no}:role/{proxy_role_name}"
    }
    resp = requests.post(endpoint,headers=get_headers(),json=body,verify=False)
    return resp.status_code,resp.text

def delete_org_mapping(domino_org_name):
    endpoint='https://irsa-svc.domino-field/delete_role_mapping'
    body={
        "domino_org":f"{domino_org_name}"
    }
    resp = requests.delete(endpoint,headers=get_headers(),json=body,verify=False)
    return resp.status_code,resp.text

In [ ]:
## Add the asset and proxy role name. Run this for all the pair
## Add aws account number
EKS_ACCOUNT_NO="<ADD_HERE>"
ASSETS_ACCOUNT_NO="<ADD_HERE>"

In [ ]:
DOMINO_ORG_NAME="irsa-iamrole-list-bucket"
ASSET_ROLE_NAME="acme-list-bucket-role"
PROXY_ROLE_NAME="acme-list-bucket-role"
add_org_to_role_mapping(EKS_ACCOUNT_NO,ASSETS_ACCOUNT_NO,DOMINO_ORG_NAME,ASSET_ROLE_NAME,PROXY_ROLE_NAME)

## Get My Mappings (All users can make this call)
```
GET https://irsa-svc.domino-field/my_role_mappings
```

It will return a list of Domino Organizations along with the corresponding AWS ROLE and the AWS Proxy Role the user is assigned to. The user can assume any of these AWS role. The actual role is assumed via the proxy aws role.

In [ ]:
#Fetch my mappings (Any user can do this)
endpoint='https://irsa-svc.domino-field/my_role_mappings'
resp = requests.get(endpoint,headers=get_headers(),verify=False)
print(resp.text)


## Get All Mappings (Only Domino Administrators can make this call)
```
GET https://irsa-svc.domino-field/role_mappings
```

It will return a list of Domino Organizations along with the corresponding AWS ROLE and the AWS Proxy Role the user is assigned to. The user can assume any of these AWS role. The actual role is assumed via the proxy aws role.

In [ ]:
endpoint='https://irsa-svc.domino-field/role_mappings'
resp = requests.get(endpoint,headers=get_headers(),verify=False)
print(resp.text)

## Delete Role Mapping (Only Domino Administrators can make this call)
```
DELETE https://irsa-svc.domino-field/delete_role_mapping
```

This endpoint adds a mapping between a Domino Organization, AWS Role (the user will assume) and the Proxy AWS Role through which the AWS role is assumed.
The payload takes the following form:
```json
{
    "domino_org":"my_test_org"
}
```


In [ ]:
domino_org_name="irsa-iamrole-list-bucket"
delete_org_mapping(domino_org_name)

## Refreshing the .aws/config file

If the mappings have changed during a live workspace, you can refresh them using the call below. There is no need to restart the workspace. 

In [ ]:
!curl http://localhost:6003/refresh

Verify if the new mappings appear in your $AWS_CONFIG_FILE

In [ ]:
!cat $AWS_CONFIG_FILE